In [1]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from time import time
import scipy.io
from numba import njit

#Local imports
import surfgeopy as sp

This is a benchmark address the computational task of computing  surface areas

We execute  HOSQ for a $T^2_{r,R}$ with inner radius $r=1$ and outer radius $R=2$. We utilized ''distmesh'', to generate Delaunay triangulations of $N_{\Delta}=2528$ triangles for the torus


In [ ]:
mesh_path ="../meshes/torus_2528.mat"

R=2
r=1
@njit(fastmath=True)
def phi(x: np.ndarray):
    ph = np.sqrt(x[0]*x[0] + x[1]*x[1])
    return (ph - R)*(ph - R) + x[2]*x[2] - r*r

@njit(fastmath=True)
def dphi(x: np.ndarray):
    ph = np.sqrt(x[0]*x[0] + x[1]*x[1])
    return np.array([-2*R*x[0]/ph + 2*x[0],-2*R*x[1]/ph + 2*x[1],2*x[2]])


def err_t(intp_degree,lp_degree,mesh_path, refinement):
    f1=lambda _: 1
    t0 = time()
    areas = sp.integration(phi,dphi, mesh_path,intp_degree,lp_degree,refinement, f1)
    t1 = time()
    sum_area =sum(areas)
    t1 = time()
    exact_area =(2*pi*R)*(2*pi*r)
    
    print("Relative error: ", abs(sum_area - exact_area) / exact_area)
    print ("The main function takes:",{(t1-t0)})
    error=abs(sum_area - exact_area) / exact_area
    return error

# here is the error obtained using Dune Curved Grid
error_dune=np.array([ 3.177343e-04, 9.601306e-05, 7.453577e-07, 2.424357e-07, 2.720754e-09,9.312086e-10, 2.317402e-11,
                         2.795971e-10, 4.334784e-09,7.391845e-08,5.396110e-07,1.843169e-06,2.399259e-05])

# Degree of Polynomial
Nrange = list(range(2,15))
lp_degree=float("inf")
refinement=0
error1=[] 
for n in Nrange:
    if n%1==0:print(n)
    erro1 = err_t(int(n),lp_degree,mesh_path,refinement)
    error1.append(erro1)

plt.semilogy(Nrange, error1, '-or')
plt.semilogy(Nrange, error_dune, '-ob')
plt.xlabel("Degree of Polynomial",fontsize=13)
plt.ylabel("Relative Error",fontsize=13)
plt.legend(['HOSQ','DCG'],prop={'size': 13},loc='upper center')
plt.xticks(np.arange(min(Nrange), max(Nrange)+1, 1.0))
plt.ylim([2.758195177427762e-17,3.9514540203871754e-03])
plt.grid()
plt.savefig("../images/convergence_for_torus_linf.pdf")

The above experiment is replicated three times, where we initiate with a rough mesh and progressively refine it twice.

In [ ]:
mesh_path ="../meshes/torus_260.mat"

R=2
r=1


@njit(fastmath=True)
def phi(x: np.ndarray):
    ph = np.sqrt(x[0]*x[0] + x[1]*x[1])
    return (ph - R)*(ph - R) + x[2]*x[2] - r*r

@njit(fastmath=True)
def dphi(x: np.ndarray):
    ph = np.sqrt(x[0]*x[0] + x[1]*x[1])
    return np.array([-2*R*x[0]/ph + 2*x[0],-2*R*x[1]/ph + 2*x[1],2*x[2]])


def err_t(intp_degree,lp_degree,mesh_path, refinement,k):
    f1=lambda _: 1
    t0 = time()
    areas = sp.integration(phi,dphi, mesh_path,intp_degree,lp_degree,refinement, f1,k)
    t1 = time()
    sum_area =sum(areas)
    t1 = time()
    exact_area =(2*pi*R)*(2*pi*r)
    
    print("Relative error: ", abs(sum_area - exact_area) / exact_area)
    print ("The main function takes:",{(t1-t0)})
    error=abs(sum_area - exact_area) / exact_area
    return error

Nrange = list(range(2,15))
lp_degree=float("inf")
error1=[] 
error2=[]
error3=[]
for n in Nrange:
    if n%1==0:print(n)
    erro1 = err_t(int(n),lp_degree,mesh_path,0,14)
    error1.append(erro1)
    erro2 = err_t(n,lp_degree,mesh_path, 1,14)
    error2.append(erro2)
    erro3 = err_t(n,lp_degree,mesh_path, 2,14)
    error3.append(erro3)

plt.semilogy(Nrange, error1, '-ok')
plt.semilogy(Nrange, error2, '-og')
plt.semilogy(Nrange, error3, '-oy')
plt.xlabel("Degree of Polynomial",fontsize=13)
plt.ylabel("Relative Error",fontsize=13)
plt.legend(['$N_{\Delta}=260$','$N_{\Delta}=1040$','$N_{\Delta}=4160$'],prop={'size': 13})
plt.xticks(np.arange(min(Nrange), max(Nrange)+1, 1.0))
plt.ylim([2.758195177427762e-18,3.9514540203871754e-03])
plt.grid()
